In [1]:
import ast

net_features = []
with open("nets.txt", "r") as f:
    for line in f:
        net = ast.literal_eval(line.strip())  # safely parse dictionary from string
        net_features.append(net)

In [ ]:
pip uninstall torch

Found existing installation: torch 2.7.0
Uninstalling torch-2.7.0:
  Would remove:
    /root/miniconda3/lib/python3.12/site-packages/torch-2.7.0.dist-info/*
    /root/miniconda3/lib/python3.12/site-packages/torch/lib/libgomp-c9fef706.so.1
    /root/miniconda3/lib/python3.12/site-packages/torch/lib/libtorch_cuda.so
    /root/miniconda3/lib/python3.12/site-packages/torch/lib/libtorch_cuda_linalg.so
    /root/miniconda3/lib/python3.12/site-packages/torch/lib/libtorch_global_deps.so
    /root/miniconda3/lib/python3.12/site-packages/torch/lib/libtorch_python.so
    /root/miniconda3/lib/python3.12/site-packages/torch/linalg/*
    /root/miniconda3/lib/python3.12/site-packages/torch/masked/*
    /root/miniconda3/lib/python3.12/site-packages/torch/monitor/*
    /root/miniconda3/lib/python3.12/site-packages/torch/mps/*
    /root/miniconda3/lib/python3.12/site-packages/torch/mtia/*
    /root/miniconda3/lib/python3.12/site-packages/torch/multiprocessing/*
    /root/miniconda3/lib/python3.12/site-p

In [4]:
import torch
print(torch.__version__)
print(dir(torch))  # Sho

AttributeError: module 'torch' has no attribute '__version__'

In [2]:
node_features = {}
for net in net_features:
    driver = net['driver']
    node_features[driver['id']] = [
        driver['x'], driver['y'], int(driver['is_fixed']), driver['area']
    ]
    for sink in net['sinks']:
        node_features[sink['id']] = [
            sink['x'], sink['y'], int(sink['is_fixed']), sink['area']
        ]

# Map node IDs to indices
sorted_node_ids = sorted(node_features.keys())
node_id_to_index = {nid: idx for idx, nid in enumerate(sorted_node_ids)}

# Node feature matrix
import torch
X = torch.tensor([node_features[nid] for nid in sorted_node_ids], dtype=torch.float)

# Create hyperedges
hyperedges = []
for net in net_features:
    node_ids = [net['driver']['id']] + [sink['id'] for sink in net['sinks']]
    edge = [node_id_to_index[nid] for nid in node_ids]
    hyperedges.append(edge)

AttributeError: module 'torch' has no attribute 'tensor'

In [ ]:
pip install torch

  Using cached torch-2.7.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (29 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached fsspec-2025.3.2-py3-none-any.whl.metadata (11 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.6.77-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.6.77-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.6.80-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.5.1.17-py3-none-manylinux_2_28_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.6.4.1-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.3.0.4-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class HyperGNN(nn.Module):
    def __init__(self, in_dim, hidden_dim):
        super(HyperGNN, self).__init__()
        self.fc1 = nn.Linear(in_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 2)  # Output: (x, y) positions

    def forward(self, x):
        x = F.relu(self.fc1(x))
        pos = self.fc2(x)
        return pos  # Shape: [num_nodes, 2]

In [ ]:
def compute_hpwl(positions, hyperedges):
    hpwl = 0.0
    for edge in hyperedges:
        x_coords = positions[edge, 0]
        y_coords = positions[edge, 1]
        hpwl += (x_coords.max() - x_coords.min()) + (y_coords.max() - y_coords.min())
    return hpwl

def compute_density_loss(positions, areas, grid_size):
    total_area = areas.sum()
    return total_area / grid_size

# Example training loop
def train(model, X, hyperedges, areas, grid_size, num_epochs=200, lr=1e-3):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    for epoch in range(num_epochs):
        model.train()
        positions = model(X)

        hpwl_loss = compute_hpwl(positions, hyperedges)
        density_loss = compute_density_loss(positions, areas, grid_size)

        loss = hpwl_loss + 0.1 * density_loss  # Weight density as secondary

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if epoch % 10 == 0:
            print(f"Epoch {epoch}: HPWL={hpwl_loss.item():.2f}, Density={density_loss.item():.4f}")

# Setup and run
in_dim = X.shape[1]
hidden_dim = 64
model = HyperGNN(in_dim, hidden_dim)

areas = torch.tensor([f[-1] for f in X], dtype=torch.float)
grid_size = 1000 * 1000  # Example total area

train(model, X, hyperedges, areas, grid_size)